In [3]:
#2026-01-15
#This is an annotated version of Andres Navarrete Image_Preprocessing pipeline (version from 2025-12-22)
#The pipeline was annotated by Guilherme Ventura using the Gemini AI

In [ ]:
#Cell 1: System Memory Check change
#This block ensures your machine has enough RAM to handle large 3D image stacks, which can often exceed several gigabytes.

In [5]:
import psutil

# Check system memory to avoid crashes during heavy image processing
vm = psutil.virtual_memory()
print("Total (seen by this process) GB:", vm.total / (1024**3))
print("Available GB:", vm.available / (1024**3))

Total (seen by this process) GB: 1023.8580017089844
Available GB: 622.9937782287598


In [6]:
# Cell 2: Imports & Library Configuration
# This cell imports the necessary tools for image math (numpy), image IO (tifffile, pims),
# and specialized microscopy algorithms(RedLionfishDeconv, WBNS).

In [7]:
import time
from WBNS import WBNS_image

import RedLionfishDeconv as rl
from tqdm import tqdm
import numpy as np
import tifffile
import scipy.ndimage as ndi
from skimage.transform import rescale, resize
from nd2reader import ND2Reader
import os
import cv2
import pims
from scipy.ndimage import gaussian_filter
from typing import Optional, Tuple

# Specialized microscopy tools
from WBNS import WBNS_image             # Wavelet Background Noise Subtraction
import RedLionfishDeconv as rl          # GPU-accelerated Richardson-Lucy Deconvolution

In [8]:
#Cell 3: Image scaling using a linear constrast stretcher
# In microscopy, images often come out "flat" or dim because the camera sensor didn't use its full dynamic range.
# This part of the code forces the image to span from a minimum value (min_val) to a maximum value (max_val), typically 0 to 65535 for 16-bit images.

In [9]:
# Get metada data tiff images
def image_scaling_intens(img, min_val, max_val, print_res=False):

    # start scaling
    img_shape = img.shape
    img_type = img.dtype
    img_min = np.amin(img)
    img_max = np.amax(img)

    if img_shape[0] <  300:
        img = np.reshape(img, newshape = -1)
        img = cv2.normalize(img, None, alpha = min_val, beta = max_val, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
        img = np.reshape(img, newshape = img_shape)
    else:
        scale = img_max - img_min
        new_scale = max_val - min_val
        img = (new_scale * (img.astype(np.float32) - img_min) / scale) + min_val

    img = img.astype(img_type.name)

    if print_res == True:
        newimg_min = np.amin(img)
        newimg_max = np.amax(img)
        print('     -Intensity Norm  from (%d , %d) to  (%d, %d) ' % (img_min, img_max, newimg_min, newimg_max))

    return img

In [10]:
# Cell 4: Reads the voxel size (so the physical size of our image in microns) from the tif file.
# Remember that this needs to be input by the user!

In [11]:
def read_tiff_voxel_size(file_path):
    """
    Extract voxel size from TIFF metadata
    """
    def _xy_voxel_size(tags, key):
        assert key in ['XResolution', 'YResolution']
        if key in tags:
            num_pixels, units = tags[key].value
            return units / num_pixels
        return 1.

    with tifffile.TiffFile(file_path) as tiff:
        image_metadata = tiff.imagej_metadata
        if image_metadata is not None:
            z = image_metadata.get('spacing', 1.)
        else:
            z = 1.

        tags = tiff.pages[0].tags
        y = _xy_voxel_size(tags, 'YResolution')
        x = _xy_voxel_size(tags, 'XResolution')

        return [x, y, z]


def read_nd2_voxel_size(image):

    md = image.metadata     # metadata object from nd2reader
    # Pixel sizes in micrometers
    x = md["pixel_microns"]
    y = md["pixel_microns"]
    z = 3.0

    return [x, y, z]

In [12]:
# Cell 5:
# This cell corrects the loss of fluorescence intensity in depth in the SPIM images, and it normalizes the intensity across the z-stack
# The code calculates a "robust statistic" for every single slice in your 3D stack.
# It doesn't use the simple mean because one very bright spot could skew the results. Instead:
# Method p95 (Default): It looks at the 95th percentile of pixels in each slice.
# This essentially says, "How bright are the actual objects in this slice, ignoring the dark background?"

In [13]:
def z_intensity_correction(
    stack: np.ndarray,
    z_axis: int = 0,
    method: str = "p95",          # "median", "p90", "p95", "p99"
    smooth_window: int = 9,       # odd number; smoothing along z
    eps: float = 1e-8,
    preserve_dtype: bool = True,
):
    """
    Multiplicative Z-intensity correction: rescales each z-slice so a chosen robust
    statistic (e.g., p95) is constant along z. Good for depth attenuation / bleaching.

    Returns corrected_stack, scale_factors (len = Z)
    """
    if stack.ndim != 3:
        raise ValueError(f"Expected 3D stack, got {stack.shape}")

    x = np.moveaxis(stack, z_axis, 0).astype(np.float32, copy=False)  # (Z, Y, X)

    # per-slice robust "level"
    if method == "median":
        levels = np.median(x.reshape(x.shape[0], -1), axis=1)
    elif method.startswith("p"):
        q = float(method[1:])
        levels = np.percentile(x.reshape(x.shape[0], -1), q, axis=1)
    else:
        raise ValueError("method must be 'median' or 'pXX' like 'p95'")

    # avoid crazy scaling if a slice is empty/dark
    levels = np.maximum(levels, eps)

    # smooth levels along z (simple moving average)
    if smooth_window is not None and smooth_window > 1:
        if smooth_window % 2 == 0:
            smooth_window += 1
        pad = smooth_window // 2
        lvl_pad = np.pad(levels, (pad, pad), mode="edge")
        kernel = np.ones(smooth_window, dtype=np.float32) / smooth_window
        levels_s = np.convolve(lvl_pad, kernel, mode="valid")
    else:
        levels_s = levels

    # target = median level (keeps overall scale similar)
    target = np.median(levels_s)
    scales = target / levels_s  # multiply each slice by scales[z]

    y = x * scales[:, None, None]

    y = np.moveaxis(y, 0, z_axis)

    if not preserve_dtype:
        return y.astype(np.float32, copy=False), scales

    # convert back
    if np.issubdtype(stack.dtype, np.integer):
        info = np.iinfo(stack.dtype)
        y = np.clip(y, info.min, info.max).astype(stack.dtype)
    else:
        y = y.astype(stack.dtype, copy=False)

    return y, scales

In [14]:
# Cell 6:
# Applies a shading correction
# It fixes a very common hardware issue in microscopy where the light isn't perfectly even across the camera sensor.
# Usually, the center of the image is bright, and the corners are slightly darker (vignetting).
# The goal is to mathematically "flatten" the background so that a cell in the corner looks just as bright as a cell in the center.
# This is done before the deconvolution because the deconvolution algorithms are very sensitive to light levels.

In [15]:
def shading_correct_xy_estimated(
    stack: np.ndarray,
    sigma_xy: float = 64.0,
    z_axis: int = 0,
    per_slice: bool = False,
    eps: float = 1e-6,
    preserve_dtype: bool = True,
):
    """
    Estimate and correct XY shading/illumination on a 3D microscopy stack.

    Model:
        corrected = img * mean(field) / field
    where `field` is a smooth illumination estimate from a large Gaussian blur.

    Parameters
    ----------
    stack : np.ndarray
        3D array (e.g., ZYX).
    sigma_xy : float
        Gaussian sigma (in pixels) used to estimate illumination field.
        Typical: 32256 depending on image size and illumination gradients.
    z_axis : int
        Axis corresponding to Z (slice dimension).
    per_slice : bool
        False (recommended): estimate ONE field from the mean projection across Z.
        True: estimate a field per slice (can adapt but may cause z-flicker).
    eps : float
        Small constant to avoid division by zero.
    preserve_dtype : bool
        If True, output dtype matches input dtype (best-effort).

    Returns
    -------
    corrected : np.ndarray
        Shading-corrected stack.
    field : np.ndarray
        The estimated 2D illumination field (Y,X) if per_slice=False,
        otherwise None (because it varies per slice).
    """

    if stack.ndim != 3:
        raise ValueError(f"Expected a 3D stack, got shape {stack.shape} (ndim={stack.ndim}).")

    in_dtype = stack.dtype
    x = np.moveaxis(stack.astype(np.float32, copy=False), z_axis, 0)  # (Z, Y, X)

    if per_slice:
        corrected = np.empty_like(x, dtype=np.float32)
        for i in range(x.shape[0]):
            field_i = gaussian_filter(x[i], sigma=sigma_xy)
            field_i = np.maximum(field_i, eps)
            norm = float(np.mean(field_i))
            corrected[i] = x[i] * (norm / field_i)
        field = None
    else:
        proj = np.mean(x, axis=0)                      # (Y, X)
        field = gaussian_filter(proj, sigma=sigma_xy)  # smooth illumination estimate
        field = np.maximum(field, eps)
        norm = float(np.mean(field))
        corrected = x * (norm / field)

    corrected = np.moveaxis(corrected, 0, z_axis)

    if not preserve_dtype:
        return corrected.astype(np.float32, copy=False), field

    # Convert back to original dtype
    if np.issubdtype(in_dtype, np.integer):
        info = np.iinfo(in_dtype)
        corrected = np.clip(corrected, info.min, info.max).astype(in_dtype)
    else:
        corrected = corrected.astype(in_dtype, copy=False)

    return corrected, field

In [16]:
# Cell 7: Local Contrast Enhancement (CLAHE), reslicing and image post-processing
# CLAHE - Contrast Limited Adaptive Histogram Equalization (CLAHE) - ANNOTATE BETTER
# Reslicing - makes the image isotropic. Our original SPIM images are anisotropic with the pixel size in XY (e.g. 0.347x0.347um)
# being much smaller than in Z (2um)
# Image post-processing - Applies background subtraction and Gaussian smoothing to "clean up" the final result.

In [ ]:
def clahe_3d_stack(
    stack: np.ndarray,
    clip_limit: float = 0.01,
    kernel_size: Optional[Tuple[int, int]] = None,
    axis: int = 0,
    preserve_dtype: bool = True,
    p_low: float = 0.5,
    p_high: float = 99.5,
    eps: float = 1e-8,
):
    """
    Slice-wise CLAHE for a 3D microscopy stack with robust normalization to [0,1].

    p_low/p_high: percentiles for intensity clipping per slice before scaling.
    """
    print("Applying clahe_3d_stack")
    from skimage import exposure

    if stack.ndim != 3:
        raise ValueError(f"Expected a 3D stack, got shape {stack.shape}")

    in_dtype = stack.dtype
    s = np.moveaxis(stack, axis, 0).astype(np.float32, copy=False)  # (S,H,W)

    out = np.empty_like(s, dtype=np.float32)

    for i in range(s.shape[0]):
        img = s[i]

        # Robust min/max from percentiles (handles outliers + weird ranges)
        lo = np.percentile(img, p_low)
        hi = np.percentile(img, p_high)

        # Avoid division by zero
        if hi <= lo + eps:
            out[i] = 0.0
            continue

        # Clip + scale to [0,1]
        img01 = np.clip(img, lo, hi)
        img01 = (img01 - lo) / (hi - lo)

        # CLAHE expects float in [0,1]
        out[i] = exposure.equalize_adapthist(
            img01,
            kernel_size=kernel_size,
            clip_limit=clip_limit,
        ).astype(np.float32, copy=False)

    out = np.moveaxis(out, 0, axis)

    if not preserve_dtype:
        return out

    # Convert back to original dtype
    if np.issubdtype(in_dtype, np.integer):
        info = np.iinfo(in_dtype)
        out = np.clip(out * info.max, 0, info.max).astype(in_dtype)
        return out

    return out.astype(in_dtype, copy=False)



def reslice(img,position,x_res,z_res):
    scale=z_res/x_res
    z,y,x=img.shape
    new_z = round(z*scale)

    #Normalize image  to [0, 1] before extrapolating
    img_max = np.amax(img).astype(np.float32)
    img_normalized = img.astype(np.float32) / img_max

    if position=='xz':
        reslice_img=np.transpose(img_normalized,[1,0,2])
        scale_img=np.zeros((y,new_z,x),dtype=np.float32)
        for i in range(y):
            scale_img[i] = resize(reslice_img[i], (new_z,x), order=3,anti_aliasing=True)

    elif position=='yz':
        reslice_img=np.transpose(img_normalized,[2,0,1])
        scale_img = np.zeros((x,new_z,y), dtype=np.float32)
        for i in range(x):
            scale_img[i] = resize(reslice_img[i], (new_z,y), order=3,anti_aliasing=True)

    elif position=='xy':
        reslice_img=np.transpose(img_normalized,[1,0,2])
        scale_img=np.zeros((y,new_z,x),dtype=np.float32)
        for i in range(y):
            scale_img[i] = resize(reslice_img[i], (new_z,x), order=3,anti_aliasing=True)
        scale_img=np.transpose(scale_img,[1,0,2])

    # Re scale intensities
    scale_img[scale_img < 0] = 0
    scale_img[scale_img > 1] = 1
    rescaled_img = (scale_img * img_max).astype(np.uint16)


    return rescaled_img



def image_postprocessing(img, resolution_px, resolution_pz, noise_lvl, sigma):
    """
    Post-processes the image by removing noise and applying smoothing, with a progress bar.
    """
    #print("image_postprocessing : image size", img.shape)

    steps = []
    if resolution_px > 0:
        steps.append("Remove Background/Noise")

    if resolution_pz > 0:
        steps.append("Remove Background/Noise z")

    if sigma > 0:
        steps.append("Gaussian Smoothing")

    pbar = tqdm(total=len(steps), desc="Postprocessing Image", unit="step")

    # Step 1: Remove background and noise
    if resolution_px > 0:
        img = WBNS_image(img, resolution_px, noise_lvl)
        pbar.update(1)

    if resolution_pz > 0:
        img_xz = np.transpose(img,[1,0,2])
        img_xz = WBNS_image(img_xz, resolution_pz, 0)
        img = np.transpose(img_xz,[1,0,2])
        pbar.update(1)

    # Step 2: Smoothing
    if sigma > 0:
        img = ndi.gaussian_filter(img, sigma)
        pbar.update(1)

    pbar.close()
    return img


def getNormalizationThresholds(img, percentiles):

    # flaten array
    if np.ndim(img) > 1:
        img = img.flatten()

    # get background thres
    low_thres = np.percentile(img,percentiles[0])
    # get foreground thres
    high_thres = np.percentile(img,percentiles[1])


    return low_thres, high_thres

def remove_outliers_image(img, low_thres, high_thres,print_res=False):

    if print_res == True:
        img_min = np.amin(img)
        img_max = np.amax(img)

    # Set the values in the image
    img[img > high_thres] = high_thres
    img = img - low_thres
    img[img < 0] = 0

    if print_res == True:
        newimg_min = np.amin(img)
        newimg_max = np.amax(img)
        print('Cropping Intensity from (%d , %d) to  (%d, %d) ' % (img_min, img_max, newimg_min, newimg_max))

    return img

In [18]:
# Cell 8: configuration and paths
# This is the section where we change the input and output folders and the different processing parameters

In [ ]:
# Utility: Print system resource usage
import subprocess
import psutil

def print_resource_usage():
    vm = psutil.virtual_memory()
    print(f"    [Resource] CPU usage: {psutil.cpu_percent()}% | RAM used: {vm.used / (1024**3):.2f} GB / {vm.total / (1024**3):.2f} GB")
    # Try to print GPU usage if nvidia-smi is available
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=utilization.gpu,memory.used,memory.total', '--format=csv,noheader,nounits'],
                               stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
        for i, line in enumerate(result.stdout.strip().split('\n')):
            util, mem_used, mem_total = line.split(',')
            print(f"    [GPU {i}] Utilization: {util.strip()}% | Memory: {mem_used.strip()} / {mem_total.strip()} MB")
    except Exception:
        print("    [Resource] GPU info: nvidia-smi not available or no GPU detected.")

     -image dimension from : (65, 256, 256) to (65, 256, 256)

Importing : C:\Users\guilherme.bastosvent\Downloads\nuclear_selection_18x.tif
- shape: (7, 51, 51) dtype: uint16
- estimated size (GB): 3.3913180232048035e-05
- voxel sizes (um): [0.34700004580400606, 0.34700004580400606, 2.0]
- image dimension : (7, 51, 51), scaling 1.0
- image dimension from : (7, 51, 51) to (7, 51, 51), scaling 1.0
shading_correct_xy_estimated
z_intensity_correction
     -image dimension from : (7, 51, 51) to (40, 51, 51) after isotropic interpolation
     -z-space from : 2.0 to 0.35
BG suntraction : 28,  28
     -Intensity Norm  from (2145 , 6571) to  (0, 65535) 
     -size(GB) :  0.005123764276504517
     -Intensity Norm  from (0 , 85425) to  (0, 65534) 
     img_xz -size(GB) :  0.005123764276504517


Postprocessing Image:   0%|          | 0/3 [00:00<?, ?step/s]c:\Users\guilherme.bastosvent\AppData\Local\miniforge3\envs\microscopy_env\lib\site-packages\joblib\externals\loky\process_executor.py:1105: UserWarning: On Windows, max_workers cannot exceed 60 due to limitations of the operating system.
  warnings.warn(
Postprocessing Image: 100%|██████████| 3/3 [00:09<00:00,  3.14s/step]


Applying clahe_3d_stack
     -Intensity Norm  from (0 , 1) to  (0, 65535) 
Elapsed Time: 26.5664 seconds, image nuclear_selection_18x.tif

Importing : C:\Users\guilherme.bastosvent\Downloads\t0120_Channel 2.tif
- shape: (80, 2304, 2304) dtype: uint16
- estimated size (GB): 0.791015625
- voxel sizes (um): [0.34700004580400606, 0.34700004580400606, 2.0]
- image dimension : (80, 2304, 2304), scaling 1.0
- image dimension from : (80, 2304, 2304) to (80, 2304, 2304), scaling 1.0
shading_correct_xy_estimated
z_intensity_correction
     -image dimension from : (80, 2304, 2304) to (461, 2304, 2304) after isotropic interpolation
     -z-space from : 2.0 to 0.3470715835140998
BG suntraction : 28,  28
     -Intensity Norm  from (43 , 52080) to  (0, 65535) 
     -size(GB) :  10.966873168945312


In [ ]:
# # Predictions for a folder

# In[3]:

# Define paths

# Images
img_src_path = r'C:\Users\andres.ortiz\Desktop\Projects\spim_preprocessing\data\input_low-res'

# output dir
outdir = r'C:\Users\andres.ortiz\Desktop\Projects\spim_preprocessing'
#outdir = r'/medicina/hmorales/projects/ImagePreprocessing/results/'

# Image data
image_scaling  = 1.0 # number > 0
xy_pixel = 0 # different than 0 to force pixel size
z_pixel = 0 # different than 0 to force pixel size

apply_clahe = True
apply_z_intensity_correction = True
apply_shading_correct = True

# normal deconvolution
psf_path = r'C:\Users\andres.ortiz\Desktop\Projects\spim_preprocessing\data\PSF_models\low_res\PSF_bw_settings_1-low_res.tif'
padding = 32
Niter = 3
Niterz = 3

# Image Normalization
min_v = 0
max_v = 65535
percentiles_source = (40, 99.99)  #99.9995 For Nuclei, 99.999 For Membranes

# BG subtraction
resolution_px0 = 10 # FWHM of the PSF in um
resolution_pz0 = 10
noise_lvl = 2

# post processing
sigma = 1.0


# Create output folder
if not os.path.exists(outdir):
    os.mkdir(outdir)

if xy_pixel > 0:
  tempScale = z_pixel / xy_pixel
else:
  tempScale = 0


if Niter > 0:
    # Open PSF and Prepare PSF
    psf = tifffile.imread(psf_path)
    psf_shape = psf.shape
    if image_scaling > 0:
        psf = rescale(psf, (image_scaling, image_scaling, image_scaling), order=3, preserve_range=True, anti_aliasing=True)
        print(f"     -image dimension from : {psf_shape} to {psf.shape}")
    psf_f = psf.astype(np.float32)
    psf = psf_f/psf_f.sum()


# In[4]:


# Get all tif images in the folder
image_names = [f for f in os.listdir(img_src_path) if f.lower().endswith(('.tif', '.tiff', '.nd2'))]

# Calculate for each model
for i, image_name in enumerate(image_names):
    print(f"\n[Step {i+1}/{len(image_names)}] Starting processing for: {image_name}")
    start_time = time.time()  # Record the start time
    print_resource_usage()

    img_path = os.path.join(img_src_path, image_name)
    print(f"  Importing : {img_path}")

    # ---------------------------------------------------------
    # Load image depending on format
    # ---------------------------------------------------------
    ext = os.path.splitext(image_name)[1].lower()

    if ext in ['.tif', '.tiff']:
        print("  Loading TIFF image...")
        img  =  tifffile.imread(img_path).astype(np.uint16)
        voxel_size = read_tiff_voxel_size(img_path)
    elif ext == '.nd2':
        print("  Loading ND2 image...")
        img = pims.open(img_path)
        voxel_size = read_nd2_voxel_size(img)
        img = np.array(img, dtype=np.uint16, copy=False)
    print("  - shape:", np.array(img).shape, "dtype:", img.dtype)
    print("  - estimated size (GB):", np.array(img).nbytes / (1024**3))
    print_resource_usage()

    physical_pixel_sizeX, physical_pixel_sizeY, physical_pixel_sizeZ = voxel_size

    if tempScale > 0:
        scale = tempScale
        physical_pixel_sizeX = xy_pixel
        physical_pixel_sizeZ = z_pixel

    print("  - voxel sizes (um):", voxel_size)

    if image_scaling > 0:
        img_shape = img.shape
        print(f"  - image dimension : {img.shape}, scaling {image_scaling}")
        img = rescale(img, (1.0, image_scaling, image_scaling), order=3, preserve_range=True, anti_aliasing=True)
        physical_pixel_sizeX /= image_scaling
        print(f"  - image dimension from : {img_shape} to {img.shape}, scaling {image_scaling}")
        img_shape = img.shape
    print_resource_usage()

    scale = physical_pixel_sizeX / physical_pixel_sizeZ

    # pre-processing
    if apply_shading_correct == True:
        print("[Check-in] Running shading_correct_xy_estimated...")
        img, field = shading_correct_xy_estimated(img, sigma_xy=96, z_axis=0, per_slice=False)
        print_resource_usage()

    if apply_z_intensity_correction == True:
        print("[Check-in] Running z_intensity_correction...")
        img, scales = z_intensity_correction(img, z_axis=0, method="p95", smooth_window=11)
        print_resource_usage()

    # Make image isotropic
    if abs(1.0-scale) > 1e-4 :
        print("[Check-in] Reslicing to isotropic...")
        img = reslice(img,'xy',physical_pixel_sizeX,physical_pixel_sizeZ)
    img = img.astype(np.float32)
    new_img_shape = img.shape
    new_physical_pixel_sizeZ = img_shape[0] * physical_pixel_sizeZ / new_img_shape[0]
    print(f"  - image dimension from : {img_shape} to {new_img_shape} after isotropic interpolation")
    print(f"  - z-space from : {physical_pixel_sizeZ} to {new_physical_pixel_sizeZ}")
    physical_pixel_sizeZ = new_physical_pixel_sizeZ
    pixel_size_X = physical_pixel_sizeX
    print_resource_usage()

    # Recalculate resolution for BG subtraction
    resolution_px = int(resolution_px0 / new_physical_pixel_sizeZ)
    resolution_pz = int(resolution_pz0 / new_physical_pixel_sizeZ)
    print(f"  BG subtraction : {resolution_px},  {resolution_pz}")

    # Deconvolution
    if Niter > 0:
        print("[Check-in] Running 3D deconvolution...")
        img = image_scaling_intens(img, min_v, max_v, True)
        img = np.pad(img, padding, mode='reflect')
        imgSizeGB = img.nbytes / (1024 ** 3)
        print('    -size(GB) : ', imgSizeGB)
        print_resource_usage()
        res_gpu = rl.doRLDeconvolutionFromNpArrays(img, psf, niter=Niter,resAsUint8=False)
        img = res_gpu[padding:-padding, padding:-padding, padding:-padding]
        print_resource_usage()

    if Niterz > 0:
        print("[Check-in] Running 2D (XZ) deconvolution...")
        img = image_scaling_intens(img, min_v, max_v, True)
        img_xz = np.transpose(img,[1,0,2])
        psf_xz = np.transpose(psf,[1,0,2])
        img_xz = np.pad(img_xz, padding, mode='reflect')
        imgSizeGB = img_xz.nbytes / (1024 ** 3)
        print('    img_xz -size(GB) : ', imgSizeGB)
        print_resource_usage()
        res_gpu = rl.doRLDeconvolutionFromNpArrays(img_xz, psf_xz, niter=Niter,resAsUint8=False)
        img_xz = res_gpu[padding:-padding, padding:-padding, padding:-padding]
        img = np.transpose(img_xz,[1,0,2])
        psf = np.transpose(psf_xz,[1,0,2])
        print_resource_usage()

    print("[Check-in] Running post-processing...")
    img = image_postprocessing(img, resolution_px, resolution_pz, noise_lvl, sigma)
    print_resource_usage()

    if apply_clahe == True:
        print("[Check-in] Applying CLAHE...")
        img_xz = np.transpose(img,[1,0,2])
        img_xz= clahe_3d_stack(img_xz, clip_limit=0.01, kernel_size=(64, 64), axis=0)
        img = np.transpose(img_xz,[1,0,2])
        print_resource_usage()

    # Image Normalization
    if percentiles_source[0] > 0 or percentiles_source[1] < 100:
        print("[Check-in] Removing outliers and normalizing intensities...")
        low_thres, high_thres = getNormalizationThresholds(img, percentiles_source)
        img = remove_outliers_image(img, low_thres, high_thres)
        print_resource_usage()

    print("[Check-in] Final intensity scaling and saving...")
    img = image_scaling_intens(img, min_v, max_v, True)
    img = img.astype(np.uint16)

    # Save images
    base_name = os.path.splitext(image_name)[0]
    image_out = f"{base_name}_{100*image_scaling}.tif"
    img_out = os.path.join(outdir, image_out)
    tifffile.imwrite(img_out,img)
    print(f"  Saved processed image to: {img_out}")

    elapsed_time = time.time() - start_time
    print(f"[Done] Elapsed Time: {elapsed_time:.4f} seconds, image {image_name}")
    print_resource_usage()